In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
benign_folder = 'benign_sites'
phishing_folder = 'phishing_sites'


In [5]:
# Function to read HTML files and assign labels
def read_html_files(folder, label):
    data = []
    for filename in os.listdir(folder):
        if filename.endswith('.html'):
            with open(os.path.join(folder, filename), 'r', encoding='utf-8') as file:
                html_content = file.read()
                data.append({'html_content': html_content, 'label': label})
    return data

# Load data from both folders
benign_data = read_html_files(benign_folder, 0)  # 0 = benign
phishing_data = read_html_files(phishing_folder, 1)  # 1 = phishing

# Combine into a DataFrame
df = pd.DataFrame(benign_data + phishing_data)


In [6]:
df.head

<bound method NDFrame.head of                                          html_content  label
0   <!DOCTYPE html>\n<html lang="en" class="webp" ...      0
1   \n    <!DOCTYPE html>\n    <html lang="en-US" ...      0
2   <!DOCTYPE html><html lang="en-us"><head><scrip...      0
3   <!DOCTYPE html>\n<html lang="en" class=" nytap...      0
4   \n\n\n\n\n\n\n\n<!doctype html>\n\n<html class...      0
5   <!DOCTYPE html><html lang="en"><head><meta cha...      0
6   <!DOCTYPE HTML>\n\n<html lang="en-US" dir="ltr...      0
7   <!DOCTYPE html>\n<html lang="en" dir="ltr" pre...      0
8   <!DOCTYPE html>\n<html lang="en" class="no-js"...      0
9   <!DOCTYPE html>\n<html lang="en" class="large-...      0
10  <!DOCTYPE html><html><head nonce="d4dd026a-2cb...      1
11  <!DOCTYPE html><html class="no-js" dir="ltr" l...      1
12  <!DOCTYPE html>\n<html lang="en">\n\n<head>\n ...      1
13  <!doctype html>\n<html lang="en">\n<head>\n  <...      1
14  <!DOCTYPE html>\n<html lang="en-US">\n  <head>...  